In [1]:
 #Import and set API keys securely
import os

In [12]:
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = userdata.get("HUGGINGFACE_API_KEY")


In [13]:
from langchain_core.documents import Document

sample_text = """
LangChain is a framework to build LLM-powered applications using components like memory, chaining, agents, and retrievers.
It allows developers to easily integrate large language models with tools, APIs, and memory to build complex reasoning applications.
"""

documents = [Document(page_content=sample_text)]


In [14]:
#splitting the documnets
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=300,
    chunk_overlap=100
)

split_docs = text_splitter.split_documents(documents)
print(f" Total Chunks: {len(split_docs)}")


 Total Chunks: 1


In [28]:
#creating embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceHubEmbeddings(
    repo_id="sentence-transformers/all-MiniLM-L6-v2",
    huggingfacehub_api_token=os.environ["HUGGINGFACE_API_KEY"]
)





In [32]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)



In [34]:
#creating vector db to store embeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    persist_directory="./chroma_db1"
)


In [35]:
#loading the retriver
retriever = vectorstore.as_retriever()


In [47]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="gemma2-9b-it",
    temperature=0
)



In [48]:
#Adding memory for conversation
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)



In [49]:
# Create RAG Chain with Conversational Memory
from langchain.chains import ConversationalRetrievalChain

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    verbose=True
)


In [50]:
print("RAG Chatbot Ready! Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print(" Goodbye!")
        break
    response = rag_chain.invoke({"question": user_input})
    print("chatBot:", response["answer"])


🤖 RAG Chatbot Ready! Type 'exit' to stop.

You: hey my name is shoeb


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
LangChain is a framework to build LLM-powered applications using components like memory, chaining, agents, and retrievers.
It allows developers to easily integrate large language models with tools, APIs, and memory to build complex reasoning applications.
Human: hey my name is shoeb

> Finished chain.

> Finished chain.
Bot: Hello Shoeb! 

It's nice to meet you.  

What can I do for you today?  


You: can you help me


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat Histor